# Equipo 2
## Miembros:
### Jorge Arturo Torres Cruz - A01176590
### Juan Manuel Pérez Font - A00819815

## Recolección de datos

In [11]:
# Para uso en colab
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# ==== FOR REFERENCE =====
# ['wines.txt', 'bubble_gums.txt', 'dumplings.txt', 'pizza_urls.txt', 'sandwich.txt']
# =========== ============
url_files = ['wines.txt', 'bubble_gums.txt', 'dumplings.txt', 'pizza_urls.txt', 'sandwich.txt']

In [4]:
# Iteramos por cada uno de los archivos y obtenemos los URLs de las imagenes.
print("Reading files containing images urls")
urls = []
for file_name in url_files:
  with open(file_name) as f:
    content = f.readlines()
    content = [url.strip() for url in content]
    print("Reading {} image urls ({})".format(len(content), file_name))
    urls.append(content)

Reading files containing images urls
Reading 1247 image urls (wines.txt)
Reading 1209 image urls (bubble_gums.txt)
Reading 1272 image urls (dumplings.txt)
Reading 1215 image urls (pizza_urls.txt)
Reading 1229 image urls (sandwich.txt)


In [0]:
# Utilizaremos urllib para descargar las imagenes utilizando URLs obtenidos de ImageNet
import urllib.request
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
import uuid
from itertools import repeat

In [10]:
def download_from_url(category_url):
  category, url = category_url
  print(f'Downloading {url} for category {category}')
  try:
    urllib.request.urlretrieve(url, f'./data/{category}/{uuid.uuid4()}.jpg')
    return url
  except Exception as e:
    print('Error')
    return f'Error: {e}'

def download_category_from_url(category, urls):
  try:
    print(f'Creating directory to store {category} images')
    os.mkdir(f'./data/{category}')
  except FileExistsError:
    print(f'./data/{category} directory exists, continuing...')
  except Exception as e:
    print(e)
  else:
    print(f'Succesfully created {category}/ directory')
  results = None
  with ThreadPoolExecutor(max_workers=5) as executor:
    return executor.map(download_from_url, zip(repeat(category), urls), timeout=2)

# ==== FOR REFERENCE =====
# ['wine','bubble_gum', 'dumplings', 'pizza', 'sandwich']
# ========================
categories = ['pizza', 'sandwich'] # wine bubble_gum
try:
  os.mkdir("./data")
except FileExistsError:
  print('data directory exists, continuing...')
for idx, category in enumerate(categories):
  download_category_from_url(category, urls[idx])

KeyboardInterrupt: ignored

## Generación de datos